## CNN sentence classifier - Round 2 of Hyperparameter tuning

Following the analysis of round 1, this is a fine tuning of the hyperparameters.

Note that data load, model creation and training have been broken out into jbyrne_utils.py to simplify the notebook.

In [1]:
## Enable Tensorboard
%load_ext tensorboard


## Usual Imports
import numpy as np

## Add local module folder to the path and load modules

import sys
sys.path.insert(0, '../python')
import debug
from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model



In [2]:
### The following uses compatibility mode to allow use
### of CUDA 11.2 with GPU support

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# maximum number of tokens to look at.
max_len = 100


### Read in the data set of sentences into numpy array.

In [4]:
d = load_data("../data/3xNCS.json")


Loaded 11056 data records.


## Split into train, validation and test sets

Before separating inputs and labels, duplicate the positive cases twice in the training data set.

In [5]:
%%time 

train_len = int(0.8 * len(d))
val_len = int(0.2 * len(d))

train, val = np.split(d, [train_len])


# split into x and y sets for use in keras runs
x_train, y_train = np.array( [ i["text"] for i in train ] ), np.array( [ i["label"] for i in train ] )
x_val,   y_val   = np.array( [ i["text"] for i in val ] ),   np.array( [ i["label"] for i in val ]   )
# x_test,  y_test  = np.array( [ i["text"] for i in test ] ),  np.array( [ i["label"] for i in test ]  )


CPU times: user 10.1 ms, sys: 4.31 ms, total: 14.4 ms
Wall time: 13.3 ms


In [6]:
## Approximately 25% of labels are positive.  To even them up, we will 

## Tokenize the training data set and build a vocabulary from it
Want to do this to minimize the vocab to just what is in the training set for performance.

using defaults:  strip all punctuation except apostrophies, set to lower case and split on spaces. No restriction on length of vocabulary.

In [7]:
# Tokenize the training data set and build a vocabulary from it
# Want to do this to minimize the vocab to just what is in
# the training set for performance.

x_train_ids, x_val_ids, x_test_ids, tokenizer = tokenize_sentences(x_train, x_val, np.array([]), max_len)


In [8]:
# Check for % of statements that are checkable claims to 
# ensure we have some level of balance that will avoid
# high accuracy for single value guessing.

print(f"proportion of checkable claims in training data  : {np.count_nonzero(y_train == 1)/len(y_train):.4f}")
print(f"proportion of checkable claims in validation data: {np.count_nonzero(y_val == 1)/len(y_val):.4f}")
# print(f"proportion of checkable claims in test data      : {np.count_nonzero(y_test == 1)/len(y_test):.4f}")

proportion of checkable claims in training data  : 0.2517
proportion of checkable claims in validation data: 0.2432


In [9]:
%%time


# number of dimensions in the embeddings
embed_dim = 50

# Get the embedding matrix for the Embedding layer in Keras.
vocab_list, embedding_matrix, vocab_size = embed_matrix(tokenizer, embed_dim)

CPU times: user 29.6 s, sys: 127 ms, total: 29.7 s
Wall time: 29.5 s


In [10]:
len( embedding_matrix)

7934

In [11]:
embedding_matrix.shape[1]

50

In [12]:
print(len(x_train_ids))
print(len(y_train))
print(len(x_val_ids))
print(len(y_val))

8844
8844
2212
2212


In [13]:
# from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model

# run_model(embedding_matrix, x_train_ids, y_train, x_val_ids, y_val)

## Output of the tests are deleted as the notebook will not load in Chrome

In [14]:
# this is very quick, so we can use grid search on these models.

run_logfile = "runs2.log"

epochs = 20  # Not a variable, so we can determine the best epoch for the best model

#Write the header line
with open(run_logfile,'a') as f:
   f.write("timestamp|max_len|epochs|batch_size|embed_dim|num_filters|kernel_sizes|dense_layer_dims|dropout_rate|")
   for metric in [ 'loss', 'accuracy', 'val_loss', 'val_accuracy']:
       f.write(f"{metric}|")
       for i in range(0,epochs):
           f.write(f"{metric}-{i}|")
   f.write(f"END\n")        
   f.close()

    
# run the models
for k_sizes in [ [8,16], [16,32], [32,64], [4,8,12], [4,8,16], [8,12,16], [8,16,32], [4,8,16,32], [4,8,32,64] ]:

    # Set filter counts according to the number of kernels in use.
    if len(k_sizes) == 2:
        filter_counts = [ [16,16], [16,32], [32,32] ]
    if len(k_sizes) == 3:
        filter_counts = [ [16,16,16], [32, 32, 32], [64, 64, 64], [8,16,32], [16,32,64] ]
    if len(k_sizes) == 4:
        filter_counts = [ [2,2,2,2], [4,4,4,4], [8,8,8,8], [32,32,32,32], [8,16,32,64] ]
    
    
    for n_fil in filter_counts:
        
        for dld in [ [8], [16], [32], [32, 16] ]:

            hist = run_model(embedding_matrix,
                                  x_train_ids,
                                  y_train,
                                  x_val_ids,
                                  y_val,
                                  kernel_sizes     = k_sizes,
                                  num_filters      = n_fil,
                                  dense_layer_dims = dld,
                                  epochs = epochs,
                                  logfile = run_logfile)
            print("\n\n\n##########################################################################\n\n\n")



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 16
Epoch 1/20
177/177 [==============================] - 3s 10ms/step - loss: 0.5618 - accuracy: 0.7499 - val_loss: 0.5130 - val_accuracy: 0.7568
Epoch 2/20
177/177 [==============================] - 1s 5ms/step - loss: 0.5090 - accuracy: 0.7574 - val_loss: 0.4483 - val_accuracy: 0.7970
Epoch 3/20
177/177 [==============================] - 1s 5ms/step - loss: 0.4098 - accuracy: 0.8199 - val_loss: 0.3851 - val_accuracy: 0.8305
Epoch 4/20
177/177 [==============================] - 1s 5ms/step - loss: 0.2919 - accuracy: 0.8807 - val_loss: 0.3613 - val_accuracy: 0.8481
Epoch 5/20
177/177 [==============================] - 1s 5ms/step - loss: 0.2239 - accuracy: 0.9139 - val_loss: 0.3556 - val_accuracy: 0.8585
Epoch 6/20
177/177 [==============================] - 1s 5ms/step - loss: 0.1583 - accuracy: 0.9418 - val_loss: 0.3872 - val_accuracy: 0.8594
Epoch 7/20
177/177 [====================

177/177 [==============================] - 1s 6ms/step - loss: 0.0264 - accuracy: 0.9913 - val_loss: 0.6393 - val_accuracy: 0.8590
Epoch 16/20
177/177 [==============================] - 1s 5ms/step - loss: 0.0224 - accuracy: 0.9917 - val_loss: 0.6941 - val_accuracy: 0.8472
Epoch 17/20
177/177 [==============================] - 1s 5ms/step - loss: 0.0207 - accuracy: 0.9930 - val_loss: 0.6707 - val_accuracy: 0.8517
Epoch 18/20
177/177 [==============================] - 1s 5ms/step - loss: 0.0207 - accuracy: 0.9923 - val_loss: 0.7741 - val_accuracy: 0.8495
Epoch 19/20
177/177 [==============================] - 1s 5ms/step - loss: 0.0175 - accuracy: 0.9938 - val_loss: 0.7719 - val_accuracy: 0.8486
Epoch 20/20
177/177 [==============================] - 1s 5ms/step - loss: 0.0192 - accuracy: 0.9929 - val_loss: 0.7496 - val_accuracy: 0.8467



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: 

177/177 [==============================] - 1s 6ms/step - loss: 0.0819 - accuracy: 0.9750 - val_loss: 0.4299 - val_accuracy: 0.8553
Epoch 9/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0559 - accuracy: 0.9826 - val_loss: 0.4625 - val_accuracy: 0.8626
Epoch 10/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0481 - accuracy: 0.9856 - val_loss: 0.5195 - val_accuracy: 0.8522
Epoch 11/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0413 - accuracy: 0.9867 - val_loss: 0.5320 - val_accuracy: 0.8653
Epoch 12/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0268 - accuracy: 0.9936 - val_loss: 0.5566 - val_accuracy: 0.8639
Epoch 13/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0266 - accuracy: 0.9926 - val_loss: 0.5594 - val_accuracy: 0.8603
Epoch 14/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0222 - accuracy: 0.9939 - val_loss: 0.6461 - val_accuracy: 0.8662
Epoch 15/20


177/177 [==============================] - 2s 9ms/step - loss: 0.6185 - accuracy: 0.7048 - val_loss: 0.4926 - val_accuracy: 0.7604
Epoch 2/20
177/177 [==============================] - 1s 6ms/step - loss: 0.4694 - accuracy: 0.7747 - val_loss: 0.4212 - val_accuracy: 0.8088
Epoch 3/20
177/177 [==============================] - 1s 6ms/step - loss: 0.3463 - accuracy: 0.8561 - val_loss: 0.3600 - val_accuracy: 0.8463
Epoch 4/20
177/177 [==============================] - 1s 6ms/step - loss: 0.2541 - accuracy: 0.9012 - val_loss: 0.3358 - val_accuracy: 0.8553
Epoch 5/20
177/177 [==============================] - 1s 6ms/step - loss: 0.1843 - accuracy: 0.9329 - val_loss: 0.3411 - val_accuracy: 0.8644
Epoch 6/20
177/177 [==============================] - 1s 6ms/step - loss: 0.1292 - accuracy: 0.9540 - val_loss: 0.3557 - val_accuracy: 0.8635
Epoch 7/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0860 - accuracy: 0.9715 - val_loss: 0.3959 - val_accuracy: 0.8666
Epoch 8/20
177/17

Epoch 16/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0227 - accuracy: 0.9931 - val_loss: 0.7821 - val_accuracy: 0.8617
Epoch 17/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0183 - accuracy: 0.9946 - val_loss: 0.7045 - val_accuracy: 0.8549
Epoch 18/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0101 - accuracy: 0.9962 - val_loss: 0.8261 - val_accuracy: 0.8621
Epoch 19/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0166 - accuracy: 0.9948 - val_loss: 0.8447 - val_accuracy: 0.8617
Epoch 20/20
177/177 [==============================] - 1s 6ms/step - loss: 0.0166 - accuracy: 0.9947 - val_loss: 0.8786 - val_accuracy: 0.8585



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 32
Adding Convolution: Kernel Size: 16, Filter Count: 32
Epoch 1/20
177/177 [==============================] - 2s 9ms/step - loss: 0.5570 - accuracy: 0.74

...

177/177 [==============================] - 1s 8ms/step - loss: 0.0703 - accuracy: 0.9786 - val_loss: 0.4359 - val_accuracy: 0.8531
Epoch 10/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0583 - accuracy: 0.9796 - val_loss: 0.4624 - val_accuracy: 0.8558
Epoch 11/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0505 - accuracy: 0.9847 - val_loss: 0.5001 - val_accuracy: 0.8599
Epoch 12/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0395 - accuracy: 0.9871 - val_loss: 0.5000 - val_accuracy: 0.8580
Epoch 13/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0355 - accuracy: 0.9895 - val_loss: 0.5305 - val_accuracy: 0.8526
Epoch 14/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0328 - accuracy: 0.9885 - val_loss: 0.5775 - val_accuracy: 0.8567
Epoch 15/20
177/177 [==============================] - 1s 7ms/step - loss: 0.0295 - accuracy: 0.9904 - val_loss: 0.6793 - val_accuracy: 0.8368
Epoch 16/20

177/177 [==============================] - 3s 11ms/step - loss: 0.5699 - accuracy: 0.7449 - val_loss: 0.5102 - val_accuracy: 0.7568
Epoch 2/20
177/177 [==============================] - 1s 8ms/step - loss: 0.5023 - accuracy: 0.7420 - val_loss: 0.4439 - val_accuracy: 0.7966
Epoch 3/20
177/177 [==============================] - 1s 8ms/step - loss: 0.4276 - accuracy: 0.8170 - val_loss: 0.3810 - val_accuracy: 0.8341
Epoch 4/20
177/177 [==============================] - 1s 8ms/step - loss: 0.2952 - accuracy: 0.8799 - val_loss: 0.3242 - val_accuracy: 0.8635
Epoch 5/20
177/177 [==============================] - 1s 8ms/step - loss: 0.2205 - accuracy: 0.9145 - val_loss: 0.3296 - val_accuracy: 0.8617
Epoch 6/20
177/177 [==============================] - 1s 8ms/step - loss: 0.1473 - accuracy: 0.9445 - val_loss: 0.3320 - val_accuracy: 0.8626
Epoch 7/20
177/177 [==============================] - 1s 8ms/step - loss: 0.1084 - accuracy: 0.9627 - val_loss: 0.3514 - val_accuracy: 0.8653
Epoch 8/20
177/1

177/177 [==============================] - 1s 8ms/step - loss: 0.0110 - accuracy: 0.9973 - val_loss: 0.6704 - val_accuracy: 0.8599
Epoch 15/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0172 - accuracy: 0.9955 - val_loss: 0.6503 - val_accuracy: 0.8612
Epoch 16/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0145 - accuracy: 0.9958 - val_loss: 0.7438 - val_accuracy: 0.8354
Epoch 17/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0147 - accuracy: 0.9960 - val_loss: 0.8242 - val_accuracy: 0.8639
Epoch 18/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0095 - accuracy: 0.9969 - val_loss: 0.7895 - val_accuracy: 0.8590
Epoch 19/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0060 - accuracy: 0.9979 - val_loss: 0.8253 - val_accuracy: 0.8522
Epoch 20/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0093 - accuracy: 0.9971 - val_loss: 0.7682 - val_accuracy: 0.8590



########

177/177 [==============================] - 1s 8ms/step - loss: 0.1734 - accuracy: 0.9350 - val_loss: 0.3450 - val_accuracy: 0.8630
Epoch 6/20
177/177 [==============================] - 1s 8ms/step - loss: 0.1197 - accuracy: 0.9574 - val_loss: 0.3686 - val_accuracy: 0.8644
Epoch 7/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0923 - accuracy: 0.9679 - val_loss: 0.3866 - val_accuracy: 0.8648
Epoch 8/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0635 - accuracy: 0.9778 - val_loss: 0.4521 - val_accuracy: 0.8603
Epoch 9/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0481 - accuracy: 0.9846 - val_loss: 0.4963 - val_accuracy: 0.8621
Epoch 10/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0346 - accuracy: 0.9895 - val_loss: 0.5042 - val_accuracy: 0.8594
Epoch 11/20
177/177 [==============================] - 1s 8ms/step - loss: 0.0253 - accuracy: 0.9922 - val_loss: 0.5704 - val_accuracy: 0.8562
Epoch 12/20
177

177/177 [==============================] - 2s 9ms/step - loss: 0.0156 - accuracy: 0.9943 - val_loss: 0.8740 - val_accuracy: 0.8630
Epoch 19/20
177/177 [==============================] - 2s 9ms/step - loss: 0.0101 - accuracy: 0.9971 - val_loss: 0.7847 - val_accuracy: 0.8594
Epoch 20/20
177/177 [==============================] - 2s 9ms/step - loss: 0.0107 - accuracy: 0.9974 - val_loss: 0.9532 - val_accuracy: 0.8409



##########################################################################









### Dataset Citations

@inproceedings{arslan2020claimbuster,
    title={{A Benchmark Dataset of Check-worthy Factual Claims}},
    author={Arslan, Fatma and Hassan, Naeemul and Li, Chengkai and Tremayne, Mark },
    booktitle={14th International AAAI Conference on Web and Social Media},
    year={2020},
    organization={AAAI}
}

@article{meng2020gradient,
  title={Gradient-Based Adversarial Training on Transformer Networks for Detecting Check-Worthy Factual Claims},
  author={Meng, Kevin and Jimenez, Damian and Arslan, Fatma and Devasier, Jacob Daniel and Obembe, Daniel and Li, Chengkai},
  journal={arXiv preprint arXiv:2002.07725},
  year={2020}
}

Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]


### Useful resources in building this
Using pre-trained word embeddings: https://keras.io/examples/nlp/pretrained_word_embeddings/